In [ ]:
!pip install torch
!pip install huggingface-hub
!pip install transformers
!pip install accelerate
!pip install langchain
!pip install chromadb
!pip install pypdf
!pip install sentence-transformers
!pip install flash-attn


# RAG (Retrieval Augmented Generation)


La generación aumentada de recuperación (RAG) es una técnica que ayuda a los modelos de lenguaje a dar respuestas más precisas y relevantes. Imagina que tienes un modelo que sabe mucho sobre un tema, pero no todo. RAG funciona así: primero, busca información en grandes bases de datos o fuentes específicas, como si fuera un bibliotecario. Luego, el modelo de lenguaje usa esa información adicional para generar respuestas más completas y detalladas. Esto es especialmente útil porque no necesitas reentrenar el modelo cada vez que hay nueva información. En lugar de eso, el modelo puede acceder a datos actualizados de Internet o a documentos internos de una empresa, haciendo que sus respuestas sean más útiles y contextualizadas.

![Descripción de la imagen](https://miro.medium.com/v2/resize:fit:1400/0*WYv0_CaBmCTt7FXc)


# Modelo de lenguaje Phi 3 128k

Documentación de Hugginface: https://huggingface.co/microsoft/Phi-3-mini-128k-instruct

El modelo Phi-3-Mini-128K-Instruct es un modelo avanzado y ligero con 3.8 mil millones de parámetros, perteneciente a la familia Phi-3. Ha sido entrenado con los conjuntos de datos Phi-3, que incluyen datos sintéticos y datos públicos filtrados, priorizando la calidad y el razonamiento profundo. Este modelo tiene dos variantes, 4K y 128K, que se refieren a la longitud del contexto en tokens que puede manejar. Después de su entrenamiento inicial, el modelo fue afinado mediante supervisión y optimización directa de preferencias para mejorar su capacidad de seguir instrucciones y cumplir con medidas de seguridad. En pruebas de sentido común, comprensión del lenguaje, matemáticas, programación, contexto a largo plazo y razonamiento lógico, el Phi-3 Mini-128K-Instruct mostró un rendimiento sólido y de vanguardia entre los modelos con menos de 13 mil millones de parámetros.

## Forma de prompt del modelo

Este modelo genera las respuestas de acuerdo al modelo del siguiente prompt

```
<|user|>
I am going to Paris, what should I see?<|end|>
<|assistant|>
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:\n\n1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.\n2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.\n3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.\n\nThese are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world."<|end|>
<|user|>
What is so great about #1?<|end|>
<|assistant|>
**Aquí modelo entrega su respuesta**
```



In [ ]:
#función para formatear prompt

def messages_to_prompt(messages: dict) -> str:
  prompt = ''
  system_found = False

  for message in messages:
    if message['role'] == 'system':
      prompt += f"<|system|>\n{message['content']}<|end|>\n"
      system_found = True
    elif message['role'] == 'assistant':
      prompt += f"<|assistant|>\n{message['content']}<|end|>\n"
    elif message['role'] == 'user':
      prompt += f"<|user|>\n{message['content']}<|end|>\n"
    else:
      prompt += f"<|user|>\n{message['content']}<|end|>\n"

  prompt += "<|assistant|>\n"

  if not system_found:
    prompt = "<|system|>\nYoy are a helpful Ai assistant and you might receive some spanish instructions, you have to response everithing in spanish. if you don't know the answer just say No encuentro la respuesta" + prompt


  return prompt

In [ ]:
message =[
    {'role':'system', 'content':'Debes responder con precisión lo que te preguntan acerca de la documentación recuperada, todo es español y muy preciso sin frases inconclusas'},
    {'role':'user', 'content':'Cuentame aspectos del Asistente'},
]
query = messages_to_prompt(message)
query

'<|system|>\nDebes responder con precisión lo que te preguntan acerca de la documentación recuperada, todo es español y muy preciso sin frases inconclusas<|end|>\n<|user|>\nCuentame aspectos del Asistente<|end|>\n<|assistant|>\n'

#Cargue de modelo mediante Hugginface y langChain

LangChain es una herramienta de código abierto que facilita el desarrollo de aplicaciones con modelos de lenguaje de gran tamaño (LLM). Piensa en LangChain como un conjunto de herramientas y APIs disponibles en Python y Javascript que te ayudan a construir fácilmente aplicaciones como chatbots y agentes virtuales. En lugar de tener que hacer todo desde cero, LangChain te proporciona los recursos necesarios para integrar estos potentes modelos de lenguaje en tus proyectos de manera más sencilla y eficiente.


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, pipeline
import transformers


import langchain
from langchain import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [ ]:
device = torch.device ("cuda" if torch.cuda.is_available() else "cpu")
model_name = "microsoft/Phi-3-mini-4k-instruct"

# se uza tokenizador del modelo LLM
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokens = tokenizer(query, padding=True, truncation=True, return_tensors='pt')
tokens

{'input_ids': tensor([[    1, 32006,  7089,   267,   620, 27582,   378, 12132,  3175,   658,
           712,   734, 16177,  1657,   273,  1274, 24340,   316,   425,  1842,
          2709, 22869,  1114, 29892, 10481,   831, 19473,   343, 12287, 12132,
         29877,  4457,  1424,  2129, 22629,  7009,   294, 32007, 32010,   315,
          8122,   420,  9565,   359,   628,  1094,   391,  2016, 32007, 32001]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}

In [ ]:
pipeline = pipeline(
    "text-generation",            # Tipo de tarea: generación de texto
    model=model_name,             # Nombre del modelo preentrenado
    tokenizer=tokenizer,          # Tokenizador a utilizar
    torch_dtype=torch.bfloat16,   # Tipo de datos a utilizar en PyTorch (bfloat16 para mejor eficiencia)
    trust_remote_code=True,       # Permite el uso de código remoto confiable
    device_map="auto",            # Asigna automáticamente los dispositivos para la ejecución (CPU/GPU)


    # Parametros para el modelo
    model_kwargs = {
        'temperature': 0.5,       # Controla la aleatoriedad de las predicciones (0.5 para un equilibrio entre coherencia y creatividad)
        'max_length': 1500,       # Longitud máxima del texto generado
        'do_sample': True,        # Activa la muestreo aleatorio durante la generación de texto
        'top_k': 10,              # Limita la consideración de las 10 mejores opciones para cada token (mejora la calidad)
        'eos_token_id': tokenizer.eos_token_id,  # Token de fin de secuencia para terminar la generación
        'attn_implementation': 'eager'
    }
)

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [ ]:
# si usas colab ejecutar de forma periodica esto, ya que si ocupas toda la vram de gpu, te cierra la sesión

torch.cuda.empty_cache()

# Procesamiento de la Data

![Descripción de la imagen](https://vectify.ai/static/images/LargeDocumentSummarization/summary.png)


## Para el ejemplo se está utilizando el procesamiento de un pdf ubicado localmente pero existen fuentes de procesamiento para obtener datos de varias fuentes

- Web Scrapping
- Data Json
- Archivos no estructurados (word, .txt, excel etc)


In [ ]:
loader = PyPDFLoader("/content/Informe Final CD.pdf")

# Aquí divide el documento por páginas
document = loader.load_and_split()

In [ ]:
len(document)

4

In [ ]:
"""
Configura el divisor de texto recursivo por caracteres.
Este divisor se utiliza para dividir textos largos en fragmentos más pequeños,

chunk_size = tamaño de la división
chunk_overlap = sobreponer sobre anteriores y/o siguientes documentos

https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/

"""

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    length_function=len,
    chunk_overlap=20
)
docs = text_splitter.split_documents(document)

In [ ]:
len(docs) #cantidad de chunks generados

40

# EMPEZANDO A ENSAMBLAR EL RAG

1. Definir el modelo de lenguaje a través del HugginfacPipeline

2. Realizar el embedding de los chunks

3. Insertar embeddings a la BD (chromaDB)

4. Realizar la cadena de Question/Anwer con langchain

In [ ]:
llm_phi3 = HuggingFacePipeline(
    pipeline = pipeline,
)

# EMBEDDING DE TEXTO PARA INGRESAR A VESTORIAL DATABASE

EMBEDDING DISPONIBLES EN LANGCHAIN
https://python.langchain.com/v0.1/docs/integrations/text_embedding/

![Descripción de la imagen](
https://miro.medium.com/v2/resize:fit:828/1*8jezi8NEx_oswGmmaqjPkw.png)

In [ ]:
embeddings = HuggingFaceEmbeddings()
vectorstore = Chroma.from_documents(docs, embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#realizando un pequeño query para obtener chunks similares

vectorstore.search("difuso", "similarity")

[Document(page_content='Problema  \nImplementar un sistema de razonam iento difuso que permita al dispositivo  detectar obstáculos  y', metadata={'page': 0, 'source': '/content/Informe Final CD.pdf'}),
 Document(page_content='aportan las del lado derecho, la diferencia entre \nestas dos aporta  el lado con menos obst áculos \npara poder girar . Loa conjuntos difus os se \nrepresentan en la figura 4 y  está  definido por', metadata={'page': 1, 'source': '/content/Informe Final CD.pdf'}),
 Document(page_content='el software  Webots, sin embargo,  para dar un ran go de salida más preciso para el controlador difuso ,', metadata={'page': 1, 'source': '/content/Informe Final CD.pdf'}),
 Document(page_content='permitir  el libre movi miento de la persona asistida.  \n \nAGENTE  de Razonamiento Difuso  \n  \nSENSORES : Los sensores utilizados propios del software', metadata={'page': 0, 'source': '/content/Informe Final CD.pdf'})]

In [ ]:
# Se emsambla el chain ingresando como parametros el Retrieval que es nuestra base de datos y nuestro modelo


qa = RetrievalQA.from_chain_type(
    llm=llm_phi3,
    chain_type = 'stuff',
    retriever = vectorstore.as_retriever(search_kwargs={'k':2}),
)

In [ ]:
# pregunta hacia el modelo
response = qa.invoke(query)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1256: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


#Respuestas para postprocesado

- Dependiendo de los parametros como temperatura o top_k, las propabilidades y creatividad del modelo harán que  entregue respuestas más precisas cortas o varias opciones. una temperatura y un top_k muy altos pueden efectuar alucionaciones más pronunciadas mientras sean menores pueden provocar respuestas menos efectivas



## Al obervar después del token ```<|assistant|>``` se podrán visualizar las respuestas del modelo


- la respuesta será un diccionario con la query y el result


In [ ]:
response

{'query': '<|system|>\nDebes responder con precisión lo que te preguntan acerca de la documentación recuperada, todo es español y muy preciso sin frases inconclusas<|end|>\n<|user|>\nCuentame aspectos del Asistente<|end|>\n<|assistant|>\n',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nwebots, fueron modificados para realizar mediciones siento \n0 el valor minimo indicando que no  hay obstaculos, y 1024 \nindicando que el obstaculo está totalmente pe gado a este.\n\notro sen sor, los o bjetos que sean angostos pueden ser detectad os al chocar con el asistente  \ny pueden ocasionar una reacción err ónea .\n\nQuestion: <|system|>\nDebes responder con precisión lo que te preguntan acerca de la documentación recuperada, todo es español y muy preciso sin frases inconclusas<|end|>\n<|user|>\nCuentame aspectos del Asistente<|end|>\n<|assistant|>\n\nHelpful Answer

In [ ]:
# Cambiar parametros del modelo sin instanciarlo de nuevo

pipeline.model.config.temperature = 1
pipeline.model.config.top_k = 2


In [ ]:
response = qa.invoke(query)
response['result']

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1256: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nwebots, fueron modificados para realizar mediciones siento \n0 el valor minimo indicando que no  hay obstaculos, y 1024 \nindicando que el obstaculo está totalmente pe gado a este.\n\notro sen sor, los o bjetos que sean angostos pueden ser detectad os al chocar con el asistente  \ny pueden ocasionar una reacción err ónea .\n\nQuestion: <|system|>\nDebes responder con precisión lo que te preguntan acerca de la documentación recuperada, todo es español y muy preciso sin frases inconclusas<|end|>\n<|user|>\nCuentame aspectos del Asistente<|end|>\n<|assistant|>\n\nHelpful Answer: El Asistente mencionado en el contexto proporcionado parece ser parte de un sistema de detección de obstáculos, posiblemente en el marco de un proyecto de robótica utilizando Webots. Este asistente puede realizar mediciones para detectar la presen

# Conclusiones

- El modelo utilizado en esta notebook ocupa al menos la mitad de vRAM que proporciona gratuitamente. No obstante de acuerdo a los recursos disponibles, en Hugginface se pueden utilizar otros modelos como LLama3 - LLama2 - Mistran ETC de 8B o 7B de paámetros. Aquí https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard se pueden viasualizar los modelos disponibles de HF de acuerdo a su cantidad de parametros y diferentes metricas.


- Langchain proporciona más cademas que realizan más procesos, por ejemplo, eslabones que resuman chunks y que el LLM responda de estos resumenes, o eslabones que limpian texto. ETC

